In [22]:
# Code to take user input and then code it in our dummy variable array
import pandas as pd
import json
import os
import numpy
import sys
from collections import namedtuple
from math import log,tan,pi,radians

In [23]:
def load_json():
    file_in = open(os.getcwd()+"/static/data/Neighborhoods_2012_polygons.json")
    d = json.load(file_in)
    return d

In [24]:
Pt = namedtuple('Pt','x,y')
Edge = namedtuple('Edge','a,b')
Poly = namedtuple('Poly','name,edges')
_eps = 1e-5
_huge = sys.float_info.max
_tiny = sys.float_info.min

In [25]:
def spherical_mercator_projection(longitude,latitude):
    x = -longitude
    y = log(tan(radians(pi/4 + latitude/2)))
    return (x,y)

In [26]:
def rayintersectseg(p, edge):
    a,b = edge
    if a.y > b.y:
        a,b = b,a
    if p.y == a.y or p.y == b.y:
        p = Pt(p.x, p.y + _eps)
    intersect = False
 
    if (p.y > b.y or p.y < a.y) or (
        p.x > max(a.x, b.x)):
        return False
 
    if p.x < min(a.x, b.x):
        intersect = True
    else:
        if abs(a.x - b.x) > _tiny:
            m_red = (b.y - a.y) / float(b.x - a.x)
        else:
            m_red = _huge
        if abs(a.x - p.x) > _tiny:
            m_blue = (p.y - a.y) / float(p.x - a.x)
        else:
            m_blue = _huge
        
        intersect = m_blue >= m_red
    return intersect

In [27]:
def is_odd(x): 
    return x%2 == 1

def ispointinside(p, poly):
    ln = len(poly)
    return is_odd(sum(rayintersectseg(p, edge)
                    for edge in poly.edges ))

In [28]:
def get_all_neighborhoods():
    d = load_json()
    shape_list=[]
    for shape_idx in range(len(d['features'])):
        name = d['features'][shape_idx]['properties']['SEC_NEIGH']

        edges =[]
        for coordinate_idx in range(len(d['features'][shape_idx]['geometry']['coordinates'][0])-1):
            lon_1 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx][0]
            lat_1 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx][1]

            lon_2 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx+1][0]
            lat_2 = d['features'][shape_idx]['geometry']['coordinates'][0][coordinate_idx+1][1]

            x1,y1 = spherical_mercator_projection(lon_1,lat_1)
            x2,y2 = spherical_mercator_projection(lon_2,lat_2)
            edges.append(Edge(a=Pt(x=x1,y=y1),b=Pt(x=x2,y=y2)))

        shape_list.append(Poly(name=name,edges=tuple(edges)))
    return shape_list

In [29]:
def find_neighborhood(test_long,test_lat,all_neighborhoods):
    x,y = spherical_mercator_projection(test_long,test_lat)
    for neighborhood in all_neighborhoods:
        correct_neighborhood = ispointinside(Pt(x=x,y=y),neighborhood)
        if correct_neighborhood:
            return neighborhood.name

all_neighborhoods = get_all_neighborhoods()

In [30]:
#41.9231° N, 87.7093° W
def get_neighborhood(long, lat):
    test_long = float(long)
    test_lat = float(lat)
    return find_neighborhood(test_long, test_lat, all_neighborhoods)

In [32]:
hoods = pd.read_csv("results_hood.csv")
hoods

,Unnamed: 0,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,...,crash_date_est_i,private_property_i,work_zone_i,work_zone_type,workers_present_i,lane_cnt,photos_taken_i,dooring_i,rd_no,hood_label
0,0,c866f3095932bfe799226e255e1079fc2bbc49bb8fe376...,2020-11-19T00:25:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MARQUETTE PARK,GAGE PARK"
1,1,725c92ece43c197a378a4f774e50ad15a4c25360ad5045...,2020-11-18T23:00:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BRIGHTON PARK,MCKINLEY PARK"
2,2,71b14bfa4ef2da35945350552503128df47a45952f0805...,2020-11-18T22:53:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,FOUR WAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HUMBOLDT PARK
3,3,79c45fc1ecbe1902dc6cef723573607eeca83d33a5d329...,2020-11-18T22:31:00.000,35,TRAFFIC SIGNAL,UNKNOWN,CLEAR,UNKNOWN,TURNING,NOT DIVIDED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GARFIELD PARK
4,4,49269ed31a112084261ca43ff3934714374c1e6aa4c36b...,2020-11-18T21:10:00.000,25,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,ONE-WAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MARQUETTE PARK,GAGE PARK"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453925,456401,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25T19:00:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC321679,LITTLE VILLAGE
453926,456402,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24T19:45:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ164689,GARFIELD PARK
453927,456403,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ122950,"MARQUETTE PARK,GAGE PARK"
453928,456404,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JB442550,GARFIELD PARK


In [33]:
hoods.drop(['Unnamed: 0'], axis=1, inplace=True)

In [34]:
hoods

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,private_property_i,work_zone_i,work_zone_type,workers_present_i,lane_cnt,photos_taken_i,dooring_i,rd_no,hood_label
0,c866f3095932bfe799226e255e1079fc2bbc49bb8fe376...,2020-11-19T00:25:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,"CURVE, LEVEL",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MARQUETTE PARK,GAGE PARK"
1,725c92ece43c197a378a4f774e50ad15a4c25360ad5045...,2020-11-18T23:00:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BRIGHTON PARK,MCKINLEY PARK"
2,71b14bfa4ef2da35945350552503128df47a45952f0805...,2020-11-18T22:53:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,FOUR WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HUMBOLDT PARK
3,79c45fc1ecbe1902dc6cef723573607eeca83d33a5d329...,2020-11-18T22:31:00.000,35,TRAFFIC SIGNAL,UNKNOWN,CLEAR,UNKNOWN,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GARFIELD PARK
4,49269ed31a112084261ca43ff3934714374c1e6aa4c36b...,2020-11-18T21:10:00.000,25,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MARQUETTE PARK,GAGE PARK"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453925,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25T19:00:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC321679,LITTLE VILLAGE
453926,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24T19:45:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ164689,GARFIELD PARK
453927,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ122950,"MARQUETTE PARK,GAGE PARK"
453928,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JB442550,GARFIELD PARK


In [35]:
test_hoods = hoods.iloc[0:1000]
test_hoods.describe

<bound method NDFrame.describe of                                        crash_record_id  \
0    c866f3095932bfe799226e255e1079fc2bbc49bb8fe376...   
1    725c92ece43c197a378a4f774e50ad15a4c25360ad5045...   
2    71b14bfa4ef2da35945350552503128df47a45952f0805...   
3    79c45fc1ecbe1902dc6cef723573607eeca83d33a5d329...   
4    49269ed31a112084261ca43ff3934714374c1e6aa4c36b...   
..                                                 ...   
995  698f5aae1249f12e7e0f191927ac7075f5edaea0f41758...   
996  aaf3a3d33399b4cec4193a58404e70c5d24b91f46af90f...   
997  e8b2ee012911034bbcb87cbd680ba740bb5118ccf280d0...   
998  7768eb9081aeb04b6cf31834dbb2c2cc97a8f277a51bbc...   
999  d0a4e9c96b84db9fbc4a5bd1aa53b8434db00d06fde338...   

                  crash_date  posted_speed_limit traffic_control_device  \
0    2020-11-19T00:25:00.000                  30         TRAFFIC SIGNAL   
1    2020-11-18T23:00:00.000                  30            NO CONTROLS   
2    2020-11-18T22:53:00.000                

In [37]:
for i in test_hoods.index:
    lat = test_hoods.at[i, 'latitude']
    long = test_hoods.at[i, 'longitude']
    test_long = float(long)
    test_lat = float(lat)
    test_hoods.at[i, 'hood_label'] = get_neighborhood(test_long, test_lat)

In [40]:
test_hoods['hood_label'].unique()

array(['SOUTH SHORE, GRAND CROSSING', 'BRIGHTON PARK,MCKINLEY PARK',
       'AUSTIN', 'GRANT PARK', 'WOODLAWN', 'AVALON PARK,CALUMET HEIGHTS',
       'HUMBOLDT PARK', 'BRONZEVILLE', 'ARCHER HEIGHTS,WEST ELSDON',
       'LINCOLN PARK', 'BACK OF THE YARDS', 'ENGLEWOOD', 'WEST RIDGE',
       'LINCOLN SQUARE', 'HYDE PARK', 'AUBURN GRESHAM', 'LOOP',
       'WEST LAWN', 'WEST PULLMAN', 'MARQUETTE PARK,GAGE PARK', 'PULLMAN',
       'WICKER PARK,WEST TOWN', 'BRIDGEPORT', 'SOUTH CHICAGO',
       'UKRAINIAN VILLAGE AND EAST VILLAGE', 'NORTH PARK,ALBANY PARK',
       'LOWER WEST SIDE', 'EDGEWATER', 'MIDWAY AIRPORT', 'UPTOWN',
       'LITTLE VILLAGE', 'WRIGLEYVILLE', 'GARFIELD PARK',
       'LITTLE ITALY, UIC', 'MOUNT GREENWOOD,MORGAN PARK', 'RIVER NORTH',
       'IRVING PARK,AVONDALE', 'DUNNING', 'LOGAN SQUARE', 'BEVERLY',
       'SOUTHEAST SIDE', 'NEAR SOUTH SIDE', 'WASHINGTON HEIGHTS,ROSELAND',
       'ROGERS PARK', None, 'NORTH LAWNDALE', 'CHATHAM,BURNSIDE',
       'PORTAGE PARK', 'LAKE VIEW',